# ML.Net - Ranking

In [12]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML" 
#r "nuget:Microsoft.ML.LightGbm" 

## Using C# Class

In [6]:
using Microsoft.ML;
using Microsoft.ML.Data;
using System;
using System.Collections.Generic;
using System.Linq;

## Declare data-classes for input data and predictions

In [7]:
public class RankingData
{
    [LoadColumn(0), ColumnName("Label")]
    public float Relevance { get; set; }

    [LoadColumn(1)]
    public float GroupId { get; set; }

    [LoadColumn(2, 133)]
    [VectorType(133)]
    public float[] Features { get; set; }
}

public class RankingPrediction
{
    [ColumnName("Score")]
    public float RelevanceScore { get; set; }
}

## Treino

In [10]:
var context = new MLContext();

var data = context.Data.LoadFromTextFile<RankingData>("./Datasets/ranking/ranking.tsv", separatorChar: '\t');

var split = context.Data.TrainTestSplit(data, testFraction: 0.2);

var secondSplit = context.Data.TrainTestSplit(split.TestSet);

//var validation = secondSplit.TrainSet;

var sampleInput = secondSplit.TestSet;

var rankingPipeline = context.Transforms.Conversion.MapValueToKey("Label")
    .Append(context.Transforms.Conversion.Hash("GroupId", "GroupId"))
    .Append(context.Ranking.Trainers.LightGbm());

var model = rankingPipeline.Fit(split.TrainSet); 

Unhandled exception: System.DllNotFoundException: Unable to load DLL 'lib_lightgbm' or one of its dependencies: Não foi possível encontrar o módulo especificado. (0x8007007E)
   at Microsoft.ML.Trainers.LightGbm.WrappedLightGbmInterface.DatasetCreateFromSampledColumn(IntPtr sampleValuePerColumn, IntPtr sampleIndicesPerColumn, Int32 numCol, Int32[] sampleNonZeroCntPerColumn, Int32 numSampleRow, Int32 numTotalRow, String parameters, SafeDataSetHandle& ret)
   at Microsoft.ML.Trainers.LightGbm.Dataset..ctor(Double[][] sampleValuePerColumn, Int32[][] sampleIndicesPerColumn, Int32 numCol, Int32[] sampleNonZeroCntPerColumn, Int32 numSampleRow, Int32 numTotalRow, String param, Single[] labels, Single[] weights, Int32[] groups)
   at Microsoft.ML.Trainers.LightGbm.LightGbmTrainerBase`4.CreateDatasetFromSamplingData(IChannel ch, Factory factory, Int32 numRow, String param, Single[] labels, Single[] weights, Int32[] groups, CategoricalMetaData catMetaData, Dataset& dataset)
   at Microsoft.M

In [12]:
var predictions = model.Transform(split.TestSet);

var options = new RankingEvaluatorOptions
{
    DcgTruncationLevel = 5
};

var metrics = context.Ranking.Evaluate(predictions, options);

var ndcg = metrics.NormalizedDiscountedCumulativeGains.Average();

Console.WriteLine($"nDGC - {ndcg}");
Console.Write(Environment.NewLine);

var batchPredictions = model.Transform(sampleInput);

var newPredictions = context.Data.CreateEnumerable<RankingPrediction>(batchPredictions, reuseRowObject: false);

Console.WriteLine("Scores:");
foreach (var prediction in newPredictions)
{
    Console.WriteLine($"{prediction.RelevanceScore}");
}

//Console.WriteLine($"Relevance - {prediction.Score}");



Unhandled exception: System.NullReferenceException: Object reference not set to an instance of an object.
   at Submission#18.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location where exception was thrown ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)